In [73]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [74]:
import galois
import numpy as np
import threading
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

np.set_printoptions(threshold=np.inf)

In [75]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [76]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [77]:
#provo co due matrici diagonali NON funziona
valori_diagonale = np.arange(1,127)
A = np.diag(valori_diagonale)

perm = np.random.permutation(126) #permutazione degli indici casuale
A_primo = A[perm, :]
A_primo = A_primo[:, perm]

A = field(A)
A_primo = field(A_primo)

In [78]:
A = field.Random((k, n_k))
#print(A)

In [79]:
id_mat = np.eye(126, dtype=int)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin1 = id_mat.copy()
matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin1 = field(matrice_bin1)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin2 = id_mat.copy()
matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin2 = field(matrice_bin2)

print(matrice_bin1)
print(matrice_bin2)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [80]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
#print(A_primo_intermedio)

In [81]:
A_primo = matrice_bin1 @ A @matrice_bin2
A_primo = field(A_primo)
#print(A_primo)

In [82]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%127)
    return somma
    #print("Riga : ", riga) #questi vanno eliminati poi
    #print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print("Somma: ", somma) #questi vanno eliminati poi
    #print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print(somme_rig)

#vado ad estrarre il primo ed il secondo elemento di ogni colonna
def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    return colonna[:4]  # Restituisce i primi quattro elementi della colonna



#eseguo in parallelo somma_riga
def esegui_somma_righe_parallelo(matrice):
    somme_rig = [0] * matrice.shape[0]  # Inizializza una lista per le somme

    with ThreadPoolExecutor() as executor:
        executor.map(somma_riga, [matrice]*matrice.shape[0], range(matrice.shape[0]))
    return somme_rig

#eseguo in parallelo estrazione
def esegui_estrazione_parallelo(matrice):
    num_colonne = matrice.shape[1]
    
    with ThreadPoolExecutor() as executor:
        #Mappo funzione estrai_da_colonna su ciascuna colonna
        risultati = list(executor.map(estrai_da_colonna, [matrice] * num_colonne, range(num_colonne)))
    
    #Separo risultati nelle rispettive liste
    primi_colonne, secondi_colonne, terzi_colonne, quarti_colonne = zip(*risultati)

    return primi_colonne, secondi_colonne, terzi_colonne, quarti_colonne

# Funzione per applicare una permutazione ciclica a un insieme di colonne
def permutazione_ciclica(matrice, blocco_dim):
    matrice_permutata = matrice.copy()
    
    # Eseguiamo una permutazione ciclica su tutte le colonne a gruppi di 'blocco_dim'
    for i in range(0, matrice.shape[1], blocco_dim):
        fine_blocco = min(i + blocco_dim, matrice.shape[1])
        # Cicla il blocco selezionato
        matrice_permutata[:, i:fine_blocco] = np.roll(matrice[:, i:fine_blocco], shift=1, axis=1)
    
    return field(matrice_permutata)


In [83]:
#fino a size 3 provato e funziona
while True:
    id_mat = np.eye(126, dtype=int)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin1 = id_mat.copy()
    matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin1 = field(matrice_bin1)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin2 = id_mat.copy()
    matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin2 = field(matrice_bin2)

    #print(matrice_bin1)
    #print(matrice_bin2)
    A_primo = matrice_bin1 @ A @ matrice_bin2
    A_primo = field(A_primo)
    #print(A_primo)"""
    somme_rig = [field(0)] * A_primo.shape[0]

    #lista per memorizzare i primi elementi della colonna
    primi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i secondi elemnti della colonna
    secondi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i terzi elementi della colonna
    terzi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i quarti elemnti della colonna
    quarti_colonne = [0] * A_primo.shape[1]

    #QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
    #PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

    colonne_finali = []
    esegui_somma_righe_parallelo(A_primo)
    #ordino in maniera crescente per valore gli indici
    indici_ordinati = np.argsort(somme_rig)
        #print(indici_ordinati)

        #applico gli swap
    A_canonica_righe = A_primo[indici_ordinati, :]
    #ora si passa alle colonne
    esegui_estrazione_parallelo(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    #qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
    A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

    #print(A_CF)

    #devo fare lo stesso per la matrice A per dimostrare che sono CF

    colonne_finali = []

    #creo un thread per ogni colonna
    esegui_somma_righe_parallelo(A)

    #print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

    #ordino in maniera crescente i valori
    indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
    A_canonica_righe = A[indici_ordinati, :]

    #print(A_canonica_righe)

    #ora si passa alle colonne
    esegui_estrazione_parallelo(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
    #print(A_CF1)

    zero = np.subtract(A_CF1,A_CF)
    zero = np.count_nonzero(zero)
    print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")
    if zero == 0:
        break


Le matrici differiscono per  619  elementi su 15876 (126*126)
Le matrici differiscono per  741  elementi su 15876 (126*126)
Le matrici differiscono per  252  elementi su 15876 (126*126)
Le matrici differiscono per  250  elementi su 15876 (126*126)
Le matrici differiscono per  738  elementi su 15876 (126*126)
Le matrici differiscono per  621  elementi su 15876 (126*126)
Le matrici differiscono per  373  elementi su 15876 (126*126)
Le matrici differiscono per  0  elementi su 15876 (126*126)


In [84]:
print("Questa è A:", A)
print("\nQuesta è A': ", A_primo)
print("\nQuesta è la forma canonica: ", A_CF)

Questa è A: [[ 52  30  70  84  92  37  88  59  67  40 113 109  15  46  62  59  49  70
   71 116  66  76  40  87  60  55  82  69  40 102  47  28 120  55  10  41
  100  21  25  28 117  12  68  31  29  63 110   7  37   4  87 108  23  92
   58 103  49  46   5 103  52  28  65  80  32  15   0  80  55  42  75  12
  110  24 121  70   5  16  44  54  27  47   1  88  92   7  51  45  14  86
   92  60 101  62   7  22 117  92   0 112  56  85  97 118 121  96 109 110
  113  27 100  93  43  79  52 112  34  21  63  87 107  92  22  62  25  94]
 [ 59  30  26 126  46  42  46  48  37 106  95  80 113  15 122  74 109  97
   17  97  31  62 117  10  51 106  30  98  80  30 110  28 103 125 120  88
   16  59  68  35  67  52 120   0  70  61 106   6 116  45  10  54 105 112
   92 107  95  59 108  59 119 100 121 104 110  54  12  69  12  79   7  31
   52 108  89  49  73  79 103 115  16  19  87  74 106 119  63  71  12  54
   74   4  64  57  16   8  38  56  81  85  88 122  47  46  61  46   6   8
   60  23  20 119  36 123

[[ 52  30  70  84  92  37  88  59  67  40 113 109  15  46  62  59  49  70
   71 116  66  76  40  87  60  55  82  69  40 102  47  28 120  55  10  41
  100  21  25  28 117  12  68  31  29  63 110   7  37   4  87 108  23  92
   58 103  49  46   5 103  52  28  65  80  32  15   0  80  55  42  75  12
  110  24 121  70   5  16  44  54  27  47   1  88  92   7  51  45  14  86
   92  60 101  62   7  22 117  92   0 112  56  85  97 118 121  96 109 110
  113  27 100  93  43  79  52 112  34  21  63  87 107  92  22  62  25  94]
 [ 59  30  26 126  46  42  46  48  37 106  95  80 113  15 122  74 109  97
   17  97  31  62 117  10  51 106  30  98  80  30 110  28 103 125 120  88
   16  59  68  35  67  52 120   0  70  61 106   6 116  45  10  54 105 112
   92 107  95  59 108  59 119 100 121 104 110  54  12  69  12  79   7  31
   52 108  89  49  73  79 103 115  16  19  87  74 106 119  63  71  12  54
   74   4  64  57  16   8  38  56  81  85  88 122  47  46  61  46   6   8
   60  23  20 119  36 123  55 123  16

In [85]:
import time

In [90]:
#Voglio vedere il tempo di esecuzione avendo già le matrici

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

colonne_finali = []
start_time = time.perf_counter()

esegui_somma_righe_parallelo(A_primo)
#ordino in maniera crescente per valore gli indici
indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]
#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

#qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

#print(A_CF)

#devo fare lo stesso per la matrice A per dimostrare che sono CF

colonne_finali = []

#creo un thread per ogni colonna
esegui_somma_righe_parallelo(A)

#print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
#print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

#print(A_canonica_righe)

#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)

for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
#print(A_CF1)
end_timer = time.perf_counter()
durata = (end_timer - start_time)*1000
print("Durata esecuzione: ", durata, " ms")

zero = np.subtract(A_CF1,A_CF)
zero = np.count_nonzero(zero)
print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")

Durata esecuzione:  342.7862339995045  ms
Le matrici differiscono per  0  elementi su 15876 (126*126)
